In [0]:
USE workspace.default

In [0]:
-- Consulta 2.1
CREATE OR REPLACE TABLE workspace.default.silver_customers AS
SELECT
  customer_id,
  nome_completo,
  CAST(data_nascimento AS DATE) AS data_nascimento,
  idade,
  cidade,
  estado,
  CAST(data_criacao_conta AS DATE) AS data_criacao_conta,
  faixa_renda_mensal,
  score_de_credito
FROM
  workspace.default.bronze_customer;

num_affected_rows,num_inserted_rows


In [0]:
-- Consulta 2.2

CREATE OR REPLACE TABLE workspace.default.silver_transactions AS
SELECT
  transaction_id,
  customer_id,
  CAST(data_transacao AS TIMESTAMP) AS data_transacao,
  tipo_transacao,
  CAST(valor_transacao AS DECIMAL(18, 2)) AS valor_transacao,
  canal,
  YEAR(CAST(data_transacao AS TIMESTAMP)) * 100 + MONTH(CAST(data_transacao AS TIMESTAMP)) AS anomes_id
FROM
  workspace.default.bronze_transactions;

num_affected_rows,num_inserted_rows


In [0]:
-- Consulta 2.3
CREATE OR REPLACE TABLE workspace.default.silver_macro_economic AS
SELECT
  CAST(data_fim_mes AS DATE) AS data_fim_mes,
  selic_fim_mes_anualizada,
  ipca_mensal,
  desemprego_mensal,
  pib_mensal,
  YEAR(CAST(data_fim_mes AS DATE)) * 100 + MONTH(CAST(data_fim_mes AS DATE)) AS anomes_id
FROM
  workspace.default.bronze_macro_economic
WHERE
  data_fim_mes IS NOT NULL;


num_affected_rows,num_inserted_rows


In [0]:
-- Cria d_date

CREATE OR REPLACE TABLE workspace.default.d_date AS
SELECT
  YEAR(datum) * 10000 + MONTH(datum) * 100 + DAY(datum) AS date_id,
  datum AS data,
  YEAR(datum) AS ano,
  MONTH(datum) AS mes,
  DAY(datum) AS dia,
  QUARTER(datum) AS trimestre,
  DAYOFWEEK(datum) AS dia_da_semana,
  DATE_FORMAT(datum, 'EEEE') AS nome_dia_da_semana,
  DATE_FORMAT(datum, 'MMMM') AS nome_mes
FROM
  (SELECT EXPLODE(SEQUENCE(MIN(CAST(data_transacao AS DATE)), MAX(CAST(data_transacao AS DATE)), INTERVAL 1 DAY)) AS datum
   FROM workspace.default.silver_transactions);

num_affected_rows,num_inserted_rows


In [0]:
-- Camada Gold

CREATE OR REPLACE TABLE workspace.default.gold_f_transactions AS
SELECT
  t.transaction_id,
  t.valor_transacao,
  t.tipo_transacao,
  t.canal,
  c.customer_id,
  d.date_id,
  m.anomes_id,
  c.idade,
  c.cidade,
  c.estado,
  c.faixa_renda_mensal,
  c.score_de_credito,
  m.selic_fim_mes_anualizada,
  m.ipca_mensal,
  m.desemprego_mensal,
  m.pib_mensal
FROM
  workspace.default.silver_transactions t
  LEFT JOIN workspace.default.silver_customers c ON t.customer_id = c.customer_id
  LEFT JOIN workspace.default.silver_macro_economic m ON t.anomes_id = m.anomes_id
  LEFT JOIN workspace.default.d_date d ON CAST(t.data_transacao AS DATE) = d.data;

num_affected_rows,num_inserted_rows
